In [14]:
import requests
import json
import csv
import numpy as np
import pandas as pd
import time
import logging
from datetime import datetime, timedelta

In [15]:
# Logging configuration
# Set the log level to INFO to display informational messages during programn execution.
logging.basicConfig(level=logging.INFO)
# A logger object is created to log events and erros in an organizzed and customized manner.
logger = logging.getLogger(__name__)

In [16]:
# API key for accesing NASA's API services.
API_KEY = "PYateWtTaDdIvDTDEy82JoVBqKaX0FBCFgmOpOiE"
# Base URL for NASA's Astronomy Picrture of the Day (APOD) API
API_URL = "https://api.nasa.gov/planetary/apod"

In [ ]:
def get_apod_data(date: str) -> dict:

    # Retry configuration 
    max_retries = 3  # Maximum number of retries
    wait_time = 10   # Time to wait between retries (seconds)
    
    for attempt in range(max_retries):
        try:
            # Construc the URL for the request
            url = f"{API_URL}?api_key={API_KEY}&date={date}"
            # Make the GET request with a 10 seconds timeout
            response = requests.get(url, timeout=10)
            # Check if the response was successful
            response.raise_for_status()
            # Check rate limit headers 
            if "X-RateLimit-Remaining" in response.headers:
                remaining = int(response.headers["X-RateLimit-Remaining"])
                if remaining == 0:
                    # Wait before retrying
                    retry_after = int(response.headers.get("Retry-After", wait_time))
                    logger.warning(f"Rate limit reached. Retrying after {retry_after} seconds...")
                    time.sleep(retry_after)
                    continue
            # Return the data in JSON format
            return response.json()
        except requests.exceptions.HTTPError as e:
            # Handle HTTP errors
            if response.status_code == 429:  # Too Many Requests
                logger.warning(f"Rate limit hit for date {date}. Retrying in {wait_time} seconds... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(wait_time)
            else:
                logger.error(f"HTTP error for date {date}: {e}")
                break
        except requests.exceptions.RequestException as e:
            # Handle request errors
            logger.error(f"Request error for date {date}: {e}")
            break
    #Return None if data retrieval failed    
    return None


In [18]:
def fetch_multiple_apod_data(start_date: str, end_date: str) -> list:
    # Initialize an empty list to store APOD data
    data = []
    try:
        # Generate a range of dates between start_date and end_date
        dates = pd.date_range(start=start_date, end=end_date)
        # Iterate over each date in the range
        for date in dates:
            # Convert the date to a string in the correct format for the API
            date_str = date.strftime("%Y-%m-%d")
            # Retrieve APOD data for the current date
            apod_data = get_apod_data(date_str)
            # Check if data retrieval was successful
            if apod_data is not None:
                # Append the data to the list 
                data.append(apod_data)
            else:
                # Log and error if data retrievalfailed
                logger.error(f"Error retrieving data for date: {date_str}")
            # Wait 5 seconds before making the next request to avoid hitting rate limits
            time.sleep(5)
    except Exception as e:
        # Log any unexpected errors that occur during the data loop
        logger.error(f"Unexpected error in date loop: {e}")
    # Return the list of APOD data
    return data


In [27]:
def save_data_to_json(data: list, filename: str) -> None:
    try:
        # Open the file in write mode
        with open(filename, 'w') as f:
            # Use json.dump to serialize the data and write it to the file
            json.dump(data, f)
        # Log a success message if the file is saved successfully
        logger.info(f"JSON file saved successfully: {filename}")
    except Exception as e:
        # Log an error message if any exception occurs during file saving
        logger.error(f"Error saving JSON file: {e}")


In [ ]:
# Parameters
start_date = "2020-01-01"  # Start date for fetching APOD data (YYYY-MM-DD)
end_date = "2020-10-27"    # End date for fetching APOD data (YYYY-MM-DD)
filename = "apod_data.json"     # Name of the JSON file to save the data

try:
    # Fetch APOD data for the spicified date range and save to JSON
    data = fetch_multiple_apod_data(start_date, end_date)
    # Check if data is not empty
    if data:
    # Save data Json file
        save_data_to_json(data, filename)
except Exception as e:
    logger.error(f"Task failed: {e}")
# Log a completion message
logger.info("Task complete")


ERROR:__main__:HTTP error for date 2020-06-10: 404 Client Error: Not Found for url: https://api.nasa.gov/planetary/apod?api_key=PYateWtTaDdIvDTDEy82JoVBqKaX0FBCFgmOpOiE&date=2020-06-10
ERROR:__main__:Error retrieving data for date: 2020-06-10
INFO:__main__:JSON file saved successfully: apod_data.json
INFO:__main__:Task complete


In [21]:
logger = logging.getLogger(__name__)

def read_apod_data(filename: str) -> dict:
    try:
        # Attemp to open and read the JSON file 
        with open(filename, 'r') as file:
            # Load the file contents int a dictionary
            data = json.load(file)
            return data
    except FileNotFoundError:
        # Handle error if file does not exist
        logger.error(f"Error: File '{filename}' not found.")
        return None
    except PermissionError:
        # Handle error if permission denied to read file
        logger.error(f"Error: Permission denied to read file '{filename}'.")
        return None
    except json.JSONDecodeError:
        # Handle error if file is empty or invalid JSON
        logger.error(f"Error: File '{filename}' is empty or invalid JSON.")
        return None
    except Exception as e:
        # Handle any another unexpected errors
        logger.error(f"Unexpected error: {e}")
        return None

def print_apod_data(data: dict) -> None:
    if data is not None:
        # Check if dictionary is not None
        for entry in data:
            # Iterate through each entry in the dictionary
            date = entry.get('date')
            title = entry.get('title')
            # Get date and title for each entry
            if date and title:
                # Check if date and title 
                logger.info(f"Date: {date}, Title: {title}")
            else:
                # Handle case if date or title is missing 
                logger.warning("Error: Date or Title missing from entry.")
    else:
        # Handle case if dictionary is None
        logger.error("Failed to load data.")
#def main ():
    # Define the filename
    #filename = 'apod_data.json'
    # Read APOD data from file
    #data = read_apod_data(filename)
    # Print APOD data
    #print_apod_data(data)

#if __name__ == '__main__':
    # Call the main function
    #main()

In [22]:
def analyze_apod_media(filename: str) -> tuple:
    try:
        # Attempt to open and read the JSON file
        with open(filename, 'r') as file:
            # Load the file contents into a dictionary 
            data = json.load(file)
    except FileNotFoundError:
        # Handle error if file does not exist
        logger.error(f"Error: File '{filename}' not found.")
        return None
    except PermissionError:
        # Hanldle error if permission denied to read file
        logger.error(f"Error: Permission denied to read file '{filename}'.")
        return None
    except json.JSONDecodeError:
        # Hanlde error of file is empty or invalid JSON
        logger.error(f"Error: File '{filename}' is empty or invalid JSON.")
        return None
    except Exception as e:
        # Handle any other unexpected errors
        logger.error(f"Unexpected error: {e}")
        return None

    # Initialize image and video cunters
    image_count = 0
    video_count = 0
    
    # Initialize variables for the date with the longest explanation
    longest_explanation_date = ""
    longest_explanation_length = 0

    # Iterate through each entry in the dictionary
    for entry in data:
        # Check if the entry contains an image or video 
        if entry.get('media_type') == 'image':
            image_count += 1
        elif entry.get('media_type') == 'video':
            video_count += 1
        
        # Check if the explanation is longer than the previous one
        explanation = entry.get('explanation')
        if explanation and len(explanation) > longest_explanation_length:
            longest_explanation_date = entry.get('date')
            longest_explanation_length = len(explanation)

    # Return the results
    return image_count, video_count, longest_explanation_date


# Show image and video count
filename = 'apod_data.json'
result = analyze_apod_media(filename)
if result:
    image_count, video_count, longest_explanation_date = result
    logger.info(f"Total images: {image_count}")
    logger.info(f"Total videos: {video_count}")
    logger.info(f"Date with longest explanation: {longest_explanation_date}")

INFO:__main__:Total images: 270
INFO:__main__:Total videos: 30
INFO:__main__:Date with longest explanation: 2020-08-31


In [ ]:
def extract_apod_data(filename: str) -> None:
    try:
        # Attempt to open and read the JSON file
        with open(filename, 'r') as file:
            # Load the file contents into a dictionary 
            data = json.load(file)
    except FileNotFoundError:
        # Handle error if file does not exist
        logger.error(f"Error: File '{filename}' not found.")
        return
    except PermissionError:
        # Handle error if permission denied to read file
        logger.error(f"Error: Permission denied to read file '{filename}'.")
        return
    except json.JSONDecodeError:
        # Handle error if file is empty or invalid JSON
        logger.info(f"Error: File '{filename}' is empty or invalid JSON.")
        return
    except Exception as e:
        # Handly any other unexpected errors
        logger.info(f"Unexpected error: {e}")
        return

    # Define the column headers for the CSV file
    headers = ['Date', 'Title', 'Media Type', 'URL']

    # Open the CSV file in append mode to avoid overwriting existing data
    with open('apod_summary.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write the column headers if the CSV file is empty
        if csvfile.tell() == 0:
            writer.writerow(headers)

        # Write each record to the CSV file
        for entry in data:
            date = entry.get('date')
            title = entry.get('title')
            media_type = entry.get('media_type')
            url = entry.get('url')
            writer.writerow([date, title, media_type, url])

    print("Data write to apod_summary.csv")

# Data success
filename = 'apod_data.json'
extract_apod_data(filename)

Data write to apod_summary.csv


In [ ]:
def create_matrix():
    # Create a 20x5 matrix with random integers between 10 and 100
    matrix = np.random.randint(10, 101, size=(20, 5))
    # Adjust the values so that the sum of each row is even
    for i in range(matrix.shape[0]):
        while np.sum(matrix[i, :]) % 2 != 0:
            # Change the last element of the row to make the sum even
            matrix[i, -1] = np.random.randint(10, 101)
            # Check if the sum is even
            if np.sum(matrix[i, :]) % 2 == 0:
                break
            # If not even, change another element of the row
            matrix[i, 0] = np.random.randint(10, 101)
    # Adjust the values so that the sum of all values is a multiple of 5
    while np.sum(matrix) % 5 != 0:
        # Change the last element of the matrix to make the sum a multiple of 5
        matrix[-1, -1] = np.random.randint(10, 101)
    return matrix

# Create and print the matrix
matrix = create_matrix()
print(matrix)

# Verify that the sum of each row is even
for i in range(matrix.shape[0]):
    assert np.sum(matrix[i, :]) % 2 == 0, f"The sum of row {i} is not even"
# Verify that the sum of all values is a multiple of 5
assert np.sum(matrix) % 5 == 0, "The sum of all values is not a multiple of 5"

[[12 37 95 51 41]
 [59 89 35 40 51]
 [70 35 41 76 22]
 [20 84 61 18 89]
 [79 64 85 59 23]
 [51 41 61 26 91]
 [36 79 11 27 57]
 [81 35 74 10 38]
 [63 95 98 26 68]
 [22 57 42 92 67]
 [38 84 39 90 15]
 [10 89 48 28 67]
 [59 85 37 32 29]
 [73 22 85 68 80]
 [78 18 63 21 38]
 [24 41 96 80 59]
 [69 84 65 14 88]
 [37 74 21 49 69]
 [24 45 55 49 69]
 [55 75 75 74 69]]


In [25]:
# Create 20x5 matrix with random integers between 10 and 100
matrix = np.random.randint(10, 101, size=(20, 5))
# Extract and print all elements from the matrix that are divisible by 3 and 5
divisible_elements = matrix[(matrix % 3 == 0) & (matrix % 5 == 0)]
print("Elements divisible by 3 y 5:")
print(divisible_elements)

# Calculate the mean of the entire matrix
matrix_mean = np.mean(matrix)
print(f"Mean of the Matrix: {matrix_mean}")

# Replace all elements in the matrix that are greater than 75 with the mean of the matrix
matrix[matrix > 75] = matrix_mean

# Print the modified matrix
print("Modified matrix:")
print(matrix)

Elements divisible by 3 y 5:
[90 15 75 15 15 30 30 75]
Mean of the Matrix: 53.77
Modified matrix:
[[24 10 53 19 57]
 [38 11 61 36 44]
 [53 53 54 36 59]
 [43 67 15 75 53]
 [53 53 53 27 53]
 [44 53 53 15 21]
 [28 47 53 38 43]
 [53 43 53 48 35]
 [49 53 51 40 51]
 [33 53 34 22 41]
 [12 53 44 63 41]
 [53 62 11 46 62]
 [53 48 46 53 47]
 [57 26 15 53 48]
 [43 30 30 11 16]
 [13 53 49 75 22]
 [53 49 53 29 53]
 [53 67 25 31 62]
 [53 69 68 36 53]
 [61 68 31 28 53]]


In [26]:
# Create a 20x5 matrix with random integers between 10 and 100
matrix = np.random.randint(10, 101, size=(20, 5))

# Calculate the mean of all values in the matrix
matrix_mean = np.mean(matrix)
print(f"Mean of the matrix: {matrix_mean}")

# Calculate the standard deviation of all values in the matrix
matrix_std_dev = np.std(matrix)
print(f"Standard deviation of the matrix: {matrix_std_dev}")

# Find the median value of the matrix
matrix_median = np.median(matrix)
print(f"Median of the matrix: {matrix_median}")

# Calculate the variance for each column of the matrix
column_variance = np.var(matrix, axis=0)
print("Variance for each column of the matrix:")
print(column_variance)

Mean of the matrix: 55.56
Standard deviation of the matrix: 25.724820699083597
Median of the matrix: 55.0
Variance for each column of the matrix:
[668.24   655.11   559.45   667.1475 636.6475]
